In [25]:
"""
Let's work on a traditional logic puzzle. 

Here's one that makes an outrageous claim right off the bat:
"""

"""
The problem:

Albert Einstein once posed a brain teaser that he predicted only 2% of the worlds population would be able to solve.

FACTS:
1. There are 5 houses in 5 different colours. 
2. In each house lives a person with a different nationality. 
3. These 5 owners drink a certain beverage, smoke a certain brand of cigarette and keep a certain pet. 
4. No owners have the same pet, brand of cigaratte, or drink.

CLUES:
1. The Brit lives in a red house 
2. The Swede keeps a dog 
3. The Dane drinks tea 
4. The green house is on the left of the white house. 
5. The green house owner drinks coffee. 
6. The person who smokes Pall Mall keeps birds. 
7. The owner of the yellow house smokes Dunhill. 
8. The man living in the house right in the center drinks milk 
9. The Norwegian lives in the first house. 
10. The man who smokes Blend lives next to the one who keeps cats 
11. The man who keeps horses lives next to the man who smokes Dunhill 
12. The owner who smokes Camel drinks beer 
13. The German smokes Marlborough. 
14. The Norwegian lives next to the blue house 
15. The man who smokes Blend has a neighbour who drinks water.

"""

"""
This is _very_ similar to sudoku, when looked at from the right angle.

"""

from logpy import run, eq, var, conde, membero
from logpy.goals import permuteq,lall, heado, tailo

In [6]:
"""
First, list out all the different classes we care about
"""

colors =        ('red', 'green', 'white', 'yellow', 'blue')
nationalities = ('british', 'swedish', 'danish', 'norwegian', 'german')
beverages =     ('tea', 'coffee', 'milk', 'beer', 'water')
smokes =        ('pall-mall', 'dunhill', 'blend', 'camel', 'marlborough')
pets =          ('dog', 'birds', 'cats', 'horses', 'fish')

rii = {}
for ii in xrange(0,5):
    rii[colors[ii]] = 0
    rii[nationalities[ii]] = 1
    rii[beverages[ii]] = 2
    rii[smokes[ii]] = 3
    rii[pets[ii]] = 4

In [13]:
"""
We'll need a way to make a house we only know a few things about
"""

def vague_house(*traits):
    house = [var(), var(), var(), var(), var()]
    for trait in traits:
        house[rii[trait]] = trait
    return tuple(house)
        
vague_house('red', 'british')

('red', 'british', ~_14, ~_15, ~_16)

In [51]:
"""
We'll need a way to talk about neighbors
"""

def neighbors(left, right, houses, ordered):
    if ordered:
        (a,b,c,d,e) = tuple(var() for x in xrange(0,5))
        
        return conde((eq(houses,(left,right,c,d,e)),),
                     (eq(houses,(a,left,right,d,e)),),
                     (eq(houses,(a,b,left,right,e)),),
                     (eq(houses,(a,b,c,left,right)),))
    else:
        return conde((neighbors(left, right, houses, True),),
                     (neighbors(right, left, houses, True),))

In [30]:
"""
The following two functions are taken from the sudoku notebook
"""

def get_rows(board):
    return tuple(board[i:i+5] for i in xrange(0,len(board),5))

def get_columns(rows):
    return tuple(tuple(map(lambda x: x[i], rows)) for i in xrange(0,5))




In [86]:
all_vars = tuple(var() for x in xrange(0,25))
houses = get_rows(all_vars)
cols = get_columns(houses)
fish_owner = var()

run(4,houses, 
    #1. The Brit lives in a red house 
    membero(vague_house('british','red'),houses),
    #2. The Swede keeps a dog 
    membero(vague_house('swedish', 'dog'), houses),
    #3. The Dane drinks tea 
    membero(vague_house('danish','tea'), houses),
    #4. The green house is on the left of the white house. 
    neighbors(vague_house('green'), vague_house('white'), houses, True),
    #5. The green house owner drinks coffee. 
    membero(vague_house('green', 'coffee'), houses),
    #6. The person who smokes Pall Mall keeps birds. 
    membero(vague_house('pall-mall', 'birds'),houses),
    #7. The owner of the yellow house smokes Dunhill. 
    membero(vague_house('yellow', 'dunhill'),houses),
    #8. The man living in the house right in the center drinks milk 
    eq(houses[2], vague_house('milk')),
    #9. The Norwegian lives in the first house. 
    conde((eq(houses[0], vague_house('norwegian')),),
          (eq(houses[4], vague_house('norwegian')),)),
    #10. The man who smokes Blend lives next to the one who keeps cats 
    neighbors(vague_house('blend'), vague_house('cats'), houses, False),
    #11. The man who keeps horses lives next to the man who smokes Dunhill 
    neighbors(vague_house('horses'),vague_house('dunhill'), houses, False),
    #12. The owner who smokes Camel drinks beer 
    membero(vague_house('camel', 'beer'),houses),
    #13. The German smokes Marlborough. 
    membero(vague_house('german','marlborough'), houses),
    #14. The Norwegian lives next to the blue house 
    neighbors(vague_house('norwegian'), vague_house('blue'), houses, False),
    #15. The man who smokes Blend has a neighbour who drinks water.
    neighbors(vague_house('blend'), vague_house('water'), houses, False),
    # This is the part that's just like sudoku
#    permuteq(colors, cols[0]),
#    permuteq(nationalities, cols[1]),
#    permuteq(beverages, cols[2]),
#    permuteq(smokes, cols[3]),
#    permuteq(pets, cols[4]),
    membero((var(),fish_owner,var(),var(),'fish'),houses)
   )

((('green', 'german', 'coffee', 'marlborough', 'fish'),
  ('white', 'swedish', 'beer', 'camel', 'dog'),
  ('red', 'british', 'milk', 'pall-mall', 'birds'),
  ('blue', 'danish', 'tea', 'blend', 'horses'),
  ('yellow', 'norwegian', 'water', 'dunhill', 'cats')),
 (('yellow', 'norwegian', 'water', 'dunhill', 'cats'),
  ('blue', 'danish', 'tea', 'blend', 'horses'),
  ('red', 'british', 'milk', 'pall-mall', 'birds'),
  ('green', 'german', 'coffee', 'marlborough', 'fish'),
  ('white', 'swedish', 'beer', 'camel', 'dog')))